# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
json_normalize(data)

,counties,info.governor,shortname,state
0,"[{'name': 'Dade', 'population': 12345}, {'name...",Rick Scott,FL,Florida
1,"[{'name': 'Summit', 'population': 1234}, {'nam...",John Kasich,OH,Ohio


In [5]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,shortname,info.governor,state
0,Dade,12345,FL,Rick Scott,Florida
1,Broward,40000,FL,Rick Scott,Florida
2,Palm Beach,60000,FL,Rick Scott,Florida
3,Summit,1234,OH,John Kasich,Ohio
4,Cuyahoga,1337,OH,John Kasich,Ohio


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
# load json as string
data_list=json.load((open('data/world_bank_projects_less.json')))

In [8]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [9]:
# load json as string
data_list=json.load((open('data/world_bank_projects.json')))

In [10]:
type(data_list)

list

In [11]:
df=json_normalize(data_list,'mjtheme_namecode',['countrycode','countryname',['_id','$oid']],record_prefix='theme_')

In [12]:
df.head()

,theme_code,theme_name,_id.$oid,countryname,countrycode
0,8,Human development,52b213b38594d8a2be17c780,Federal Democratic Republic of Ethiopia,ET
1,11,,52b213b38594d8a2be17c780,Federal Democratic Republic of Ethiopia,ET
2,1,Economic management,52b213b38594d8a2be17c781,Republic of Tunisia,TN
3,6,Social protection and risk management,52b213b38594d8a2be17c781,Republic of Tunisia,TN
4,5,Trade and integration,52b213b38594d8a2be17c782,Tuvalu,TV


In [13]:
#Find the 10 countries with most projects, using project id as the unique identifier for each project

df.groupby('countryname')['_id.$oid'].nunique().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: _id.$oid, dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 5 columns):
theme_code     1499 non-null object
theme_name     1499 non-null object
_id.$oid       1499 non-null object
countryname    1499 non-null object
countrycode    1499 non-null object
dtypes: object(5)
memory usage: 58.6+ KB


In [15]:
#find the top 10 major project themes (using column 'mjtheme_namecode')
result=df[['theme_code','theme_name']]
result=result[df['theme_name']!='']
result['theme_name'].value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Name: theme_name, dtype: int64

In [16]:
#display all theme_code and theme_name pair so we know each theme_name's corresponding theme_code
map_dict=result.drop_duplicates()
map_dict

,theme_code,theme_name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [23]:
df=pd.merge(map_dict,df[['theme_code','_id.$oid','countryname','countrycode']],on='theme_code')

df.head(10)

,theme_code,theme_name,_id.$oid,countryname,countrycode
0,8,Human development,52b213b38594d8a2be17c780,Federal Democratic Republic of Ethiopia,ET
1,8,Human development,52b213b38594d8a2be17c787,People's Republic of China,CN
2,8,Human development,52b213b38594d8a2be17c791,Republic of Madagascar,MG
3,8,Human development,52b213b38594d8a2be17c792,Kingdom of Cambodia,KH
4,8,Human development,52b213b38594d8a2be17c792,Kingdom of Cambodia,KH
5,8,Human development,52b213b38594d8a2be17c792,Kingdom of Cambodia,KH
6,8,Human development,52b213b38594d8a2be17c797,Republic of Tajikistan,TJ
7,8,Human development,52b213b38594d8a2be17c797,Republic of Tajikistan,TJ
8,8,Human development,52b213b38594d8a2be17c79a,Lao People's Democratic Republic,LA
9,8,Human development,52b213b38594d8a2be17c79e,People's Republic of Angola,AO
